### Choose the first entry of each patient. Extract body Glasgow coma scale data into ***Glasgow***, create ***Glasgow_index*** for each patient. 
Contains patient Glasgow coma scale data from both nurseCharting.csv

**1. Extract sub-categories patient id from cardiovascular**

In [30]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import data_toolbox

# change to your folder path
os.chdir('C:/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1')
# os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/DataExtract/data') 


# import diagnosis.csv
df_diagnosis = pd.read_csv('diagnosis.csv')
df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'], inplace=True)

# select cardiovascular patients
df_cardiovascular = df_diagnosis[df_diagnosis['diagnosisstring'].str.contains('cardiovascular')]

# get shock patient 
shock_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('shock')]

# get ventricular patient 
ventricular_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('ventricular')]

# get chest pain patient 
chest_pain_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('chest pain')]

# get arrhythmias patient 
arrhythmias_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('arrhythmias')]

# put id together
df_wanted = pd.concat([shock_patient, ventricular_patient, chest_pain_patient, arrhythmias_patient])

# Get the patient ids from df_wanted & sort the patient id
# patient_id_all multiple entry patient's stayid
patient_id_all = df_wanted['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

[ 143870  151179  151900 ... 3351297 3352230 3352231]


**2. Exclude patient whose unitvisitnumbe>1**

In [31]:
# import patient.csv
df_patient = pd.read_csv('patient.csv')
df_patient.sort_values(by=['patientunitstayid'], inplace=True)
df_patient_buf = df_patient[df_patient['patientunitstayid'].isin(patient_id_all)]
df_1time_patient = df_patient_buf[df_patient_buf['unitvisitnumber']==1]
# print(df_1time_patient)

# select the patient id from df_1time_patient
patient_id = df_1time_patient['patientunitstayid'].unique()
print(f'Total number of patients: {len(patient_id)}')

Total number of patients: 915


**3. Extract data of patients within the id list**

In [32]:
# import nurseCharting.csv
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)

# select the wanted patient
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# print the shape of the wanted file
print(f'nurseCharting shape: {df_nurseCharting.shape}')

nurseCharting shape: (603027, 8)


**4. Extract Heart Rate data & create index**

In [33]:
# nursingchartcelltypevallabel Temperature
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Glasgow coma score']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'Glasgow score'})
print(df_nurseCharting.head())
print(df_nurseCharting.shape)

     nursingchartid  patientunitstayid  observationoffset  \
90        265165334             143870                  3   
199       265197766             143870                239   
178       120775807             143870                433   
175        84619753             143870                763   
82        174911019             143870                962   

     nursingchartentryoffset nursingchartcelltypecat  \
90                         3                  Scores   
199                      239                  Scores   
178                      433                  Scores   
175                      763                  Scores   
82                       962                  Scores   

    nursingchartcelltypevallabel nursingchartcelltypevalname Glasgow score  
90            Glasgow coma score                   GCS Total            15  
199           Glasgow coma score                   GCS Total            15  
178           Glasgow coma score                   GCS Total     

In [34]:
# extract glasgow score from df_nurseCharting
Glasgow = df_nurseCharting[['patientunitstayid', 'observationoffset', 'Glasgow score']].copy()
Glasgow.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of Temp: \n{Glasgow.head()} \n')

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(Glasgow['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(Glasgow))
# create first occurrence index for every patient
Glasgow_index = pd.Series(first_occurrences)
print(f'First 5 rows of Temp_index: \n{Glasgow_index.head()}')

First 5 rows of Temp: 
     patientunitstayid  observationoffset Glasgow score
90              143870                  3            15
199             143870                239            15
178             143870                433            15
175             143870                763            15
82              143870                962            15 

First 5 rows of Temp_index: 
0     0
1     5
2    19
3    39
4    42
dtype: int64


**Example: how to use Glasgow & Glasgow_index**

In [35]:
# if we want the i th patient's data (i starts from 0)
# use Glasgow.iloc[Glasgow_index[i]:Glasgow_index[i+1]]
i = 0
print(f'Glasgow score data for patient {i+1}: \n{Glasgow.iloc[Glasgow_index[i]:Glasgow_index[i+1]]}')

Glasgow score data for patient 1: 
     patientunitstayid  observationoffset Glasgow score
90              143870                  3            15
199             143870                239            15
178             143870                433            15
175             143870                763            15
82              143870                962            15
